# Import CSV

In [2]:
import pandas as pd
from datetime import datetime

df = pd.read_csv(".\\InputData\\Natural Gas Futures Historical Data 1997 - 2021.csv")

# print(df.head(10))

# Format Dates

In [3]:
from datetime import datetime

df["Date"] = df["Date"].apply(lambda x: datetime.strptime(x, "%b %d, %Y"))

# print(df.head(10))

# Convert Volumes to float

In [4]:
def convert_volume(vol_string):
    if vol_string == "-":
        return 0
    
    return 1000 * float(vol_string.replace("K", ""))

    
df["Vol."] = df["Vol."].apply(convert_volume)

# print(df.head(10))

# Arrange/Format Headers

In [5]:
df = df.drop(columns=["Change %"])
df = df.rename(columns={"Price": "Close"})
df = df.rename(columns={"Vol.": "Volume"})

df = df[["Date", "Open", "High", "Low", "Close", "Volume"]]

print(df.head())

        Date   Open   High    Low  Close   Volume
0 2021-04-23  52.15  52.55  49.40  50.72  25590.0
1 2021-04-22  53.25  54.71  53.00  53.86   7710.0
2 2021-04-21  52.82  53.29  52.20  52.82   5300.0
3 2021-04-20  52.00  53.64  51.52  52.82   9320.0
4 2021-04-19  52.00  53.10  51.85  52.48   6600.0


# Trying out TA-lib

In [6]:
import pandas_ta as ta

df.set_index("Date", inplace=True)
df.sort_index(inplace=True)

# df.ta.log_return(cumulative=True, append=True)
# df.ta.percent_return(cumulative=True, append=True)
# df.ta.sma(length=10, append=True)

# df["GC"] = df.ta.sma(50, append=True) > df.ta.sma(200, append=True)
# df.ta.tsignals(df.GC, asBool=True, append=True)

all_strategies = df.ta.strategy(ta.CommonStrategy)

print(df.tail())

             Open   High    Low  Close   Volume  SMA_10   SMA_20   SMA_50  \
Date                                                                        
2021-04-19  52.00  53.10  51.85  52.48   6600.0  49.163  47.9715  45.2814   
2021-04-20  52.00  53.64  51.52  52.82   9320.0  49.548  48.4575  45.3744   
2021-04-21  52.82  53.29  52.20  52.82   5300.0  50.034  48.8120  45.3686   
2021-04-22  53.25  54.71  53.00  53.86   7710.0  50.700  49.1715  45.4512   
2021-04-23  52.15  52.55  49.40  50.72  25590.0  51.086  49.3350  45.5074   

             SMA_200  VOL_SMA_20  
Date                              
2021-04-19  38.71545      7642.0  
2021-04-20  38.90190      7758.0  
2021-04-21  39.09065      7669.0  
2021-04-22  39.28740      7583.0  
2021-04-23  39.47265      8361.5  
